In [33]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
import json
import tweepy
import io
import os
from dotenv import find_dotenv, load_dotenv
import time

In [34]:
df_tae = pd.read_csv('twitter-archive-enhanced.csv')

df_tae.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
196,854732716440526848,NaN,NaN,2017-04-19 16:25:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Marlee. She fetched a flower and immed...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/854732716...,12,10,Marlee,NaN,NaN,NaN,NaN
1229,713761197720473600,NaN,NaN,2016-03-26 16:15:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Piper. She would really like that tenn...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/713761197...,12,10,Piper,NaN,NaN,NaN,NaN
814,771014301343748096,NaN,NaN,2016-08-31 15:58:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Another pic without a dog in it? What am I sup...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/771014301...,7,10,NaN,NaN,NaN,NaN,NaN
1807,676916996760600576,NaN,NaN,2015-12-16 00:09:23 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Super speedy pupper. Does not go gentle into t...,NaN,NaN,NaN,https://vine.co/v/imJ0BdZOJTw,10,10,NaN,NaN,NaN,pupper,NaN
215,851224888060895234,NaN,NaN,2017-04-10 00:06:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Snoop. His number one passion is sticking...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/851224888...,13,10,Snoop,NaN,NaN,NaN,NaN


In [35]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

page = requests.get(url)
df_img = pd.read_csv(io.StringIO(page.content.decode('utf-8')), sep='\t')

with open('image-predictions.tsv', 'wb') as file:
    file.write(page.content)

df_img.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
794,690932576555528194,https://pbs.twimg.com/media/CZavgf4WkAARpFM.jpg,1,snorkel,0.526536,False,muzzle,0.048089,False,scuba_diver,0.034226,False
1375,763103485927849985,https://pbs.twimg.com/media/CpcWknPXYAAeLP9.jpg,2,seat_belt,0.685821,False,ice_bear,0.081597,False,chow,0.039085,True
1472,779834332596887552,https://pbs.twimg.com/media/CtKHLuCWYAA2TTs.jpg,1,golden_retriever,0.993830,True,cocker_spaniel,0.003143,True,Great_Pyrenees,0.000917,True
368,672968025906282496,https://pbs.twimg.com/media/CVbc2V2WsAE3-kn.jpg,1,toy_poodle,0.678046,True,miniature_poodle,0.160273,True,Airedale,0.065649,True
130,668291999406125056,https://pbs.twimg.com/media/CUZABzGW4AE5F0k.jpg,1,web_site,0.995535,False,skunk,0.001363,False,badger,0.000686,False


In [36]:
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)

    
# Load credentials from environment variables
bearer_token = os.getenv('BEARER_TOKEN')
consumer_key = os.getenv('CONSUMER_KEY')
consumer_secret = os.getenv('CONSUMER_SECRET')
access_token = os.getenv('ACCESS_TOKEN')
access_secret = os.getenv('ACCESS_SECRET')

client = tweepy.Client(bearer_token=bearer_token,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_secret,
    wait_on_rate_limit=True
    )

In [37]:
tweet_ids = df_img['tweet_id'].tolist()

# Process in batches of 100 so it works for the get_tweets method
with open('tweet_json.txt', 'a') as file:
    for i in range(0, len(tweet_ids), 100):
        try:
            tweets = client.get_tweets(tweet_ids[i:i+100], tweet_fields=['public_metrics','text'])
            for tweet in tweets.data:
                if tweet.data:
                    file.write(json.dumps(tweet.data) + '\n')
                else:
                    print(f"No data returned for {tweet.id}")
        
        except tweepy.TooManyRequests as e:
            print("Rate limit reached. Retrying after 15 mins and 5 secs...")
            time.sleep(15 * 61)  # Wait 15 minutes and 5 secs.

        except Exception as e:
            print("An error occurred: " + str(e))

KeyboardInterrupt: 